三种结构类型:

- 命名字段(named-field)结构体
- 类元组(tuple-like)结构体
- 类单元(unit-like)结构体

# 命名字段结构体

In [2]:
struct GrayscaleMap {
    pixels: Vec<u8>,
    size: (usize, usize)
}

In [3]:
impl GrayscaleMap {
    fn new_map(size: (usize, usize), pixels: Vec<u8>) -> GrayscaleMap {
        assert_eq!(pixels.len(), size.0 * size.1);
        GrayscaleMap{pixels, size}
    }
}

In [6]:
let width = 1024;
let height = 576;
let image = GrayscaleMap {
    pixels: vec![0; width * height],
    size: (width, height)
};

assert_eq!(image.size, (1024, 576));
assert_eq!(image.pixels.len(), 1024 * 576);

In [15]:
#[derive(Copy, Clone, Debug)]
enum BroomIntent {FetchWater, DumpWater}

#[derive(Debug)]
struct Broom {
    name: String,
    height: u32,
    health: u32,
    position: (f32, f32, f32),
    intent: BroomIntent
}

In [17]:
fn chop(b: Broom) -> (Broom, Broom) {
    let mut broom1 = Broom {height: b.height / 2, ..b};
    let mut broom2 = Broom {name: broom1.name.clone(), ..broom1};
    
    broom1.name.push_str(" I");
    broom2.name.push_str(" II");    
    
    (broom1, broom2)
}

let hokey = Broom {
    name: "Hoken".to_string(),
    height: 60,
    health: 100,
    position: (100.0, 200.0, 0.0),
    intent: BroomIntent::FetchWater
};

println!("{:?}", hokey);


Broom { name: "Hoken", height: 60, health: 100, position: (100.0, 200.0, 0.0), intent: FetchWater }


# 类元组结构体

In [23]:
#[derive(Debug)]
struct Bounds(usize, usize);

In [25]:
let image_bounds = Bounds(1024, 768);
println!("{:?}", image_bounds);

Bounds(1024, 768)


In [26]:
// newtype
struct Ascii(Vec<u8>);

# 类单元结构体

In [30]:
#[derive(Debug)]
struct Onesuch;

let o = Onesuch;

println!("{:?}", o);

Onesuch


# 定义方法

In [39]:
struct Queue {
    older: Vec<char>,
    younger: Vec<char>
}

impl Queue {
    fn new() -> Queue {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
    
    fn push(&mut self, c: char) {
        self.younger.push(c);
    }
    
    fn pop(&mut self) -> Option<char> {
        if self.older.is_empty() {
            if self.younger.is_empty() {
                return None;
            }

            use std::mem::swap;
            swap(&mut self.older, &mut self.younger);
            self.older.reverse();
        }

        self.older.pop()
    }
    
    fn is_empty(&self) -> bool {
        self.older.is_empty() && self.younger.is_empty()
    }
    
    fn split(self) -> (Vec<char>, Vec<char>) { // move
        (self.older, self.younger)
    }
}

In [38]:
let mut q = Queue { older: Vec::new(), younger: Vec::new() };

q.push('0');
q.push('1');
assert_eq!(q.pop(), Some('0'));
assert_eq!(q.pop(), Some('1'));
assert_eq!(q.pop(), None);

assert!(q.is_empty());

q.push('P');
q.push('D');

let (older, younger) = q.split();
println!("{:?}", older);
println!("{:?}", younger);

[]
['P', 'D']


# 泛型结构体

In [40]:
struct Queue<T> {
    older: Vec<T>,
    younger: Vec<T>
}

In [41]:
impl<T> Queue<T> {
    fn new() -> Queue<T> {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
    
    fn push(&mut self, t: T) {
        self.younger.push(t);
    }
    
    fn is_emplty(&self) -> bool {
        self.older.is_empty() && self.younger.is_empty()
    }
}

In [42]:
let mut q = Queue::<char>::new();
q.push('C');

let mut r = Queue::new();
r.push(0.74);

# 带生命周期参数的结构体

In [47]:
struct Extrema<'elt> {
    greatest: &'elt i32, // 结构体中有引用字段时, 必须要有生命周期参数
    least: &'elt i32
}

In [50]:
fn find_extrema<'s>(slice: &'s [i32]) -> Extrema<'s> {
    let mut greatest = &slice[0];
    let mut least = &slice[0];
    
    for i in 1..slice.len() {
        if slice[i] < *least {
            least = &slice[i];
        }
        if slice[i] > *greatest {
            greatest = &slice[i];
        }
    }
    
    Extrema {greatest, least}
}

In [54]:
{
    let a = [0, -3, 0, 15, 48];
    let e = find_extrema(&a);
    assert_eq!(-3, *e.least);
    assert_eq!(48, *e.greatest);
    
    println!("done.");
}

done.


()

# 继承通用特质

In [56]:
#[derive(Copy, Clone, Debug, PartialEq)]
struct Point {
    x: f64,
    y: f64
}

# 内部可变性

In [66]:
use std::cell::Cell;
use std::fs::File;

struct SpiderRobot {
    species: String,
    web_enabled: bool,
    leg_devices: Vec<File>,
    hardware_error_count: Cell<u32>              // Cell类型
}

impl SpiderRobot {
    fn add_hardware_error(&self) {
        let n = self.hardware_error_count.get(); // 读
        self.hardware_error_count.set(n+1);      // 写
    }
    
    fn has_hardware_errors(&self) -> bool {
        self.hardware_error_count.get() > 0      // 读
    }
}

In [68]:
let robot = SpiderRobot {
    species: "demo".to_string(),
    web_enabled: false,
    leg_devices: Vec::new(),
    hardware_error_count: Cell::new(0)
};

robot.add_hardware_error();
println!("{}", robot.has_hardware_errors());

true


In [80]:
use std::cell::RefCell;

let ref_cell: RefCell<String> = RefCell::new("hello".to_string());

{
    let r = ref_cell.borrow();
    let count = r.len();
    assert_eq!(count, 5);
    assert_eq!("hello", *r);
}

{
    let mut w = ref_cell.borrow_mut();
    w.push_str("world");
    assert_eq!("helloworld", *w);
}

()